In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_13.npz')
test_data = np.load('./Data2Vec_Large/test_13.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-09-20 16:26:25.039645: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 16:26:25.648542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-20 16:26:27.055248: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-20 16:26:27.055273: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: mojo
2024-09-20 16:26:27.055278: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: mojo
2024-09-20 16:26:27.055398: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.183.1
2024-09-20 16:26:27.055416: I external/local_xla/xla/stream_executor/

22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 469ms/step - accuracy: 0.5293 - loss: 1.3172 - val_accuracy: 0.4244 - val_loss: 0.7308
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 461ms/step - accuracy: 0.6412 - loss: 0.6639 - val_accuracy: 0.5872 - val_loss: 0.6719
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.6754 - loss: 0.6258 - val_accuracy: 0.6453 - val_loss: 0.6087
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 463ms/step - accuracy: 0.7782 - loss: 0.5046 - val_accuracy: 0.6686 - val_loss: 0.6195
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 461ms/step - accuracy: 0.7924 - loss: 0.4454 - val_accuracy: 0.7326 - val_loss: 0.5395
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 463ms/step - accuracy: 0.8330 - loss: 0.3542 - val_accuracy: 0.7500 - val_loss: 0.4892
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.9042 - loss: 0.2339 - val_accuracy: 0.7442 - val_loss: 0.5314
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.9393 - loss: 0.1785 - val_accuracy: 0.784

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5380 - loss: 1.2525 - val_accuracy: 0.5756 - val_loss: 0.8343
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5580 - loss: 1.4477 - val_accuracy: 0.5930 - val_loss: 0.7705
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5241 - loss: 1.4427 - val_accuracy: 0.5814 - val_loss: 0.7194
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5716 - loss: 1.0947 - val_accuracy: 0.6047 - val_loss: 0.6546
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5958 - loss: 0.9733 - val_accuracy: 0.5988 - val_loss: 0.6568
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5826 - loss: 0.8483 - val_accuracy: 0.5872 - val_loss: 0.6781
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5502 - loss: 0.9099 - val_accuracy: 0.6453 - val_loss: 0.6400
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6119 - loss: 0.7181 - val_accuracy: 0.6744 - val_loss: 0.6210
Epo

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_14.npz')
test_data = np.load('./Data2Vec_Large/test_14.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 463ms/step - accuracy: 0.5558 - loss: 1.8632 - val_accuracy: 0.4186 - val_loss: 0.7002
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 455ms/step - accuracy: 0.5611 - loss: 0.6840 - val_accuracy: 0.5581 - val_loss: 0.6907
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 459ms/step - accuracy: 0.6651 - loss: 0.6494 - val_accuracy: 0.5640 - val_loss: 0.6827
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.6850 - loss: 0.5863 - val_accuracy: 0.6686 - val_loss: 0.6101
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 455ms/step - accuracy: 0.7723 - loss: 0.5210 - val_accuracy: 0.7151 - val_loss: 0.5342
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.7793 - loss: 0.4602 - val_accuracy: 0.7151 - val_loss: 0.5144
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 461ms/step - accuracy: 0.8677 - loss: 0.3483 - val_accuracy: 0.7733 - val_loss: 0.5091
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.8890 - loss: 0.2639 - val_accuracy: 0.761

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5645 - loss: 1.2659 - val_accuracy: 0.5756 - val_loss: 0.8635
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5378 - loss: 1.1294 - val_accuracy: 0.6221 - val_loss: 0.7827
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5629 - loss: 1.0140 - val_accuracy: 0.6163 - val_loss: 0.6489
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5853 - loss: 0.9372 - val_accuracy: 0.5349 - val_loss: 0.7068
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5803 - loss: 0.9453 - val_accuracy: 0.6105 - val_loss: 0.6684
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5812 - loss: 0.8013 - val_accuracy: 0.6163 - val_loss: 0.6604
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6047 - loss: 0.8004 - val_accuracy: 0.6163 - val_loss: 0.6431
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6345 - loss: 0.7114 - val_accuracy: 0.6628 - val_loss: 0.6341
Ep

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_15.npz')
test_data = np.load('./Data2Vec_Large/test_15.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 464ms/step - accuracy: 0.5426 - loss: 1.5972 - val_accuracy: 0.5523 - val_loss: 0.6882
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.6450 - loss: 0.6577 - val_accuracy: 0.6686 - val_loss: 0.6176
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.7266 - loss: 0.5458 - val_accuracy: 0.6453 - val_loss: 0.6256
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.7766 - loss: 0.5060 - val_accuracy: 0.7093 - val_loss: 0.5906
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.7834 - loss: 0.4568 - val_accuracy: 0.6744 - val_loss: 0.6800
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 463ms/step - accuracy: 0.8942 - loss: 0.2849 - val_accuracy: 0.7384 - val_loss: 0.5506
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 460ms/step - accuracy: 0.8822 - loss: 0.2601 - val_accuracy: 0.7209 - val_loss: 0.5869
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 461ms/step - accuracy: 0.9405 - loss: 0.1652 - val_accuracy: 0.691

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5240 - loss: 1.3641 - val_accuracy: 0.5698 - val_loss: 0.7944
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5893 - loss: 1.2289 - val_accuracy: 0.6279 - val_loss: 0.6827
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5845 - loss: 1.0969 - val_accuracy: 0.5930 - val_loss: 0.6862
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6041 - loss: 0.9680 - val_accuracy: 0.6105 - val_loss: 0.6483
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5972 - loss: 0.8961 - val_accuracy: 0.6221 - val_loss: 0.6414
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6478 - loss: 0.7261 - val_accuracy: 0.6337 - val_loss: 0.6284
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6410 - loss: 0.7738 - val_accuracy: 0.6279 - val_loss: 0.6422
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6388 - loss: 0.7311 - val_accuracy: 0.6047 - val_loss: 0.6150
Epo

In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_16.npz')
test_data = np.load('./Data2Vec_Large/test_16.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 456ms/step - accuracy: 0.5211 - loss: 2.5679 - val_accuracy: 0.4186 - val_loss: 0.6942
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 447ms/step - accuracy: 0.5424 - loss: 0.6843 - val_accuracy: 0.5814 - val_loss: 0.6751
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 449ms/step - accuracy: 0.6537 - loss: 0.6758 - val_accuracy: 0.6105 - val_loss: 0.6589
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 450ms/step - accuracy: 0.6850 - loss: 0.5967 - val_accuracy: 0.7384 - val_loss: 0.5498
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.7902 - loss: 0.4845 - val_accuracy: 0.7907 - val_loss: 0.4788
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.8117 - loss: 0.4144 - val_accuracy: 0.7907 - val_loss: 0.4366
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 451ms/step - accuracy: 0.8443 - loss: 0.3583 - val_accuracy: 0.8314 - val_loss: 0.3918
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 450ms/step - accuracy: 0.8887 - loss: 0.2588 - val_accuracy: 0.854

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5599 - loss: 1.4757 - val_accuracy: 0.4942 - val_loss: 0.7591
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5773 - loss: 1.0063 - val_accuracy: 0.5756 - val_loss: 0.6771
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5573 - loss: 0.9952 - val_accuracy: 0.6047 - val_loss: 0.6479
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5750 - loss: 0.9139 - val_accuracy: 0.6570 - val_loss: 0.6380
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6005 - loss: 0.8006 - val_accuracy: 0.5988 - val_loss: 0.6728
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6394 - loss: 0.7691 - val_accuracy: 0.6802 - val_loss: 0.6242
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6833 - loss: 0.6870 - val_accuracy: 0.6802 - val_loss: 0.5831
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7200 - loss: 0.6062 - val_accuracy: 0.7442 - val_loss: 0.5417
Epo

In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_17.npz')
test_data = np.load('./Data2Vec_Large/test_17.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 453ms/step - accuracy: 0.5260 - loss: 1.7255 - val_accuracy: 0.4419 - val_loss: 0.6894
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 441ms/step - accuracy: 0.6172 - loss: 0.6463 - val_accuracy: 0.7558 - val_loss: 0.4966
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 448ms/step - accuracy: 0.7965 - loss: 0.4492 - val_accuracy: 0.8314 - val_loss: 0.4388
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 447ms/step - accuracy: 0.8553 - loss: 0.3583 - val_accuracy: 0.8372 - val_loss: 0.3846
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 442ms/step - accuracy: 0.8832 - loss: 0.2938 - val_accuracy: 0.8488 - val_loss: 0.3592
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 445ms/step - accuracy: 0.9330 - loss: 0.1968 - val_accuracy: 0.8605 - val_loss: 0.2853
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 444ms/step - accuracy: 0.9201 - loss: 0.1878 - val_accuracy: 0.8663 - val_loss: 0.3354
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 445ms/step - accuracy: 0.9523 - loss: 0.1328 - val_accuracy: 0.901

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5304 - loss: 1.5157 - val_accuracy: 0.6395 - val_loss: 0.7799
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5767 - loss: 1.1541 - val_accuracy: 0.6744 - val_loss: 0.6513
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6144 - loss: 1.0496 - val_accuracy: 0.7849 - val_loss: 0.4989
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6640 - loss: 0.9232 - val_accuracy: 0.7791 - val_loss: 0.4591
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7091 - loss: 0.7046 - val_accuracy: 0.7733 - val_loss: 0.4670
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7395 - loss: 0.6537 - val_accuracy: 0.8198 - val_loss: 0.4206
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7366 - loss: 0.5578 - val_accuracy: 0.7791 - val_loss: 0.4681
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7770 - loss: 0.5455 - val_accuracy: 0.8140 - val_loss: 0.4024
Epo

In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_18.npz')
test_data = np.load('./Data2Vec_Large/test_18.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 469ms/step - accuracy: 0.5513 - loss: 2.6029 - val_accuracy: 0.7326 - val_loss: 0.5962
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 459ms/step - accuracy: 0.7660 - loss: 0.5320 - val_accuracy: 0.8023 - val_loss: 0.4462
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.8444 - loss: 0.3987 - val_accuracy: 0.8314 - val_loss: 0.3960
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.8601 - loss: 0.3315 - val_accuracy: 0.8314 - val_loss: 0.3276
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.9145 - loss: 0.2280 - val_accuracy: 0.8198 - val_loss: 0.3182
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.9328 - loss: 0.1925 - val_accuracy: 0.8488 - val_loss: 0.4111
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 459ms/step - accuracy: 0.9418 - loss: 0.1454 - val_accuracy: 0.8547 - val_loss: 0.3612
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.9686 - loss: 0.0841 - val_accuracy: 0.877

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5694 - loss: 1.0572 - val_accuracy: 0.5988 - val_loss: 0.6316
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5742 - loss: 1.0818 - val_accuracy: 0.6279 - val_loss: 0.6692
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6455 - loss: 0.9054 - val_accuracy: 0.7791 - val_loss: 0.5428
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6928 - loss: 0.7045 - val_accuracy: 0.7849 - val_loss: 0.5216
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7070 - loss: 0.6056 - val_accuracy: 0.7733 - val_loss: 0.4855
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7624 - loss: 0.5772 - val_accuracy: 0.8198 - val_loss: 0.4274
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7806 - loss: 0.5369 - val_accuracy: 0.8314 - val_loss: 0.4005
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8082 - loss: 0.4740 - val_accuracy: 0.8372 - val_loss: 0.4092
Epo

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_19.npz')
test_data = np.load('./Data2Vec_Large/test_19.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-09-21 23:35:35.111205: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-21 23:35:36.097834: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=act

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 463ms/step - accuracy: 0.5273 - loss: 3.0028 - val_accuracy: 0.6279 - val_loss: 0.6796
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 462ms/step - accuracy: 0.6638 - loss: 0.6539 - val_accuracy: 0.6977 - val_loss: 0.6175
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 466ms/step - accuracy: 0.7600 - loss: 0.5685 - val_accuracy: 0.7500 - val_loss: 0.5250
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 469ms/step - accuracy: 0.8156 - loss: 0.4259 - val_accuracy: 0.7384 - val_loss: 0.5407
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 471ms/step - accuracy: 0.8324 - loss: 0.3959 - val_accuracy: 0.7384 - val_loss: 0.5663
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 469ms/step - accuracy: 0.8527 - loss: 0.3862 - val_accuracy: 0.7965 - val_loss: 0.4722
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 469ms/step - accuracy: 0.8487 - loss: 0.3568 - val_accuracy: 0.7733 - val_loss: 0.5223
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 468ms/step - accuracy: 0.9174 - loss: 0.2161 - val_accu

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5190 - loss: 1.3288 - val_accuracy: 0.4942 - val_loss: 0.8082
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6266 - loss: 1.1251 - val_accuracy: 0.6395 - val_loss: 0.6733
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6006 - loss: 1.1123 - val_accuracy: 0.6977 - val_loss: 0.5972
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5950 - loss: 1.0113 - val_accuracy: 0.7209 - val_loss: 0.5876
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6535 - loss: 0.7785 - val_accuracy: 0.7384 - val_loss: 0.5714
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6510 - loss: 0.7938 - val_accuracy: 0.6337 - val_loss: 0.6362
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6710 - loss: 0.7564 - val_accuracy: 0.7267 - val_loss: 0.5657
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7058 - loss: 0.6407 - val_accuracy: 0.7093 - val_loss: 0.5600
Epo

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_20.npz')
test_data = np.load('./Data2Vec_Large/test_20.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 462ms/step - accuracy: 0.4909 - loss: 2.4372 - val_accuracy: 0.5581 - val_loss: 0.6933
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.6442 - loss: 0.6499 - val_accuracy: 0.6221 - val_loss: 0.6560
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.7431 - loss: 0.5504 - val_accuracy: 0.6570 - val_loss: 0.6184
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.8284 - loss: 0.4400 - val_accuracy: 0.6802 - val_loss: 0.5549
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.8485 - loss: 0.3851 - val_accuracy: 0.7326 - val_loss: 0.5536
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.8596 - loss: 0.3670 - val_accuracy: 0.7616 - val_loss: 0.5653
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.8961 - loss: 0.2800 - val_accuracy: 0.7500 - val_loss: 0.5318
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.9265 - loss: 0.2244 - val_accuracy: 0.773

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4822 - loss: 1.3440 - val_accuracy: 0.5523 - val_loss: 0.7068
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5334 - loss: 1.1456 - val_accuracy: 0.5814 - val_loss: 0.7132
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5951 - loss: 1.0270 - val_accuracy: 0.5349 - val_loss: 0.6891
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5712 - loss: 0.9154 - val_accuracy: 0.5291 - val_loss: 0.7326
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5923 - loss: 0.8253 - val_accuracy: 0.6860 - val_loss: 0.6545
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5635 - loss: 0.7947 - val_accuracy: 0.5872 - val_loss: 0.6669
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6142 - loss: 0.7071 - val_accuracy: 0.7035 - val_loss: 0.6382
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5966 - loss: 0.7857 - val_accuracy: 0.5930 - val_loss: 0.6659
Ep

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_21.npz')
test_data = np.load('./Data2Vec_Large/test_21.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 465ms/step - accuracy: 0.5088 - loss: 1.9648 - val_accuracy: 0.4593 - val_loss: 0.6944
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 451ms/step - accuracy: 0.6086 - loss: 0.6747 - val_accuracy: 0.4535 - val_loss: 0.7003
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.5780 - loss: 0.6661 - val_accuracy: 0.6860 - val_loss: 0.6407
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.6460 - loss: 0.6140 - val_accuracy: 0.6453 - val_loss: 0.6300
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.7610 - loss: 0.5510 - val_accuracy: 0.7035 - val_loss: 0.5870
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.7457 - loss: 0.5294 - val_accuracy: 0.6802 - val_loss: 0.5569
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.8165 - loss: 0.4075 - val_accuracy: 0.7326 - val_loss: 0.5721
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 455ms/step - accuracy: 0.8217 - loss: 0.4003 - val_accuracy: 0.703

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4404 - loss: 1.5729 - val_accuracy: 0.5814 - val_loss: 0.8872
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5128 - loss: 1.3817 - val_accuracy: 0.4942 - val_loss: 0.7675
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5527 - loss: 1.0041 - val_accuracy: 0.5116 - val_loss: 0.7240
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5529 - loss: 1.0280 - val_accuracy: 0.6279 - val_loss: 0.6655
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5012 - loss: 0.9396 - val_accuracy: 0.4767 - val_loss: 0.7041
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6002 - loss: 0.8376 - val_accuracy: 0.5465 - val_loss: 0.6801
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5703 - loss: 0.8088 - val_accuracy: 0.5465 - val_loss: 0.6933
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6021 - loss: 0.7563 - val_accuracy: 0.5465 - val_loss: 0.6781
Epo

In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_22.npz')
test_data = np.load('./Data2Vec_Large/test_22.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 475ms/step - accuracy: 0.5021 - loss: 3.3481 - val_accuracy: 0.4244 - val_loss: 0.6934
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 469ms/step - accuracy: 0.5685 - loss: 0.6865 - val_accuracy: 0.4244 - val_loss: 0.6936
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 469ms/step - accuracy: 0.6102 - loss: 0.6722 - val_accuracy: 0.5233 - val_loss: 0.6923
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 473ms/step - accuracy: 0.6200 - loss: 0.6761 - val_accuracy: 0.5116 - val_loss: 0.6941
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 473ms/step - accuracy: 0.6291 - loss: 0.6388 - val_accuracy: 0.6279 - val_loss: 0.6423
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 467ms/step - accuracy: 0.6895 - loss: 0.5921 - val_accuracy: 0.5698 - val_loss: 0.6844
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 471ms/step - accuracy: 0.7063 - loss: 0.5504 - val_accuracy: 0.6105 - val_loss: 0.6595
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 473ms/step - accuracy: 0.7722 - loss: 0.4810 - val_accuracy: 0.622

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5660 - loss: 1.2221 - val_accuracy: 0.5349 - val_loss: 0.7539
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5825 - loss: 1.1768 - val_accuracy: 0.5640 - val_loss: 0.6701
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5416 - loss: 1.2027 - val_accuracy: 0.5640 - val_loss: 0.7157
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5662 - loss: 0.9843 - val_accuracy: 0.5000 - val_loss: 0.7239
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5607 - loss: 0.8800 - val_accuracy: 0.5058 - val_loss: 0.6972
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5547 - loss: 0.8252 - val_accuracy: 0.4884 - val_loss: 0.6958
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5569 - loss: 0.7421 - val_accuracy: 0.5581 - val_loss: 0.6701
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6061 - loss: 0.6737 
ANN Test Accuracy: 0.5736
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms

In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_23.npz')
test_data = np.load('./Data2Vec_Large/test_23.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 484ms/step - accuracy: 0.5425 - loss: 1.1607 - val_accuracy: 0.4128 - val_loss: 0.6985
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 478ms/step - accuracy: 0.5604 - loss: 0.6857 - val_accuracy: 0.4012 - val_loss: 0.7060
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 479ms/step - accuracy: 0.6035 - loss: 0.6682 - val_accuracy: 0.5349 - val_loss: 0.7070
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 483ms/step - accuracy: 0.6392 - loss: 0.6345 - val_accuracy: 0.5756 - val_loss: 0.6731
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 482ms/step - accuracy: 0.7012 - loss: 0.6105 - val_accuracy: 0.5581 - val_loss: 0.7029
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 476ms/step - accuracy: 0.6847 - loss: 0.5859 - val_accuracy: 0.5988 - val_loss: 0.6747
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 477ms/step - accuracy: 0.7836 - loss: 0.5035 - val_accuracy: 0.6221 - val_loss: 0.7145
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 480ms/step - accuracy: 0.8029 - loss: 0.4355 - val_accuracy: 0.587

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4636 - loss: 1.5424 - val_accuracy: 0.4942 - val_loss: 0.7930
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4797 - loss: 1.4581 - val_accuracy: 0.4942 - val_loss: 0.8762
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5272 - loss: 1.0797 - val_accuracy: 0.4244 - val_loss: 0.7487
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5546 - loss: 0.8995 - val_accuracy: 0.4826 - val_loss: 0.7109
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5395 - loss: 0.8864 - val_accuracy: 0.4302 - val_loss: 0.7133
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5245 - loss: 0.8203 - val_accuracy: 0.3953 - val_loss: 0.7021
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4938 - loss: 0.7862 - val_accuracy: 0.4186 - val_loss: 0.7032
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5781 - loss: 0.7964 - val_accuracy: 0.4477 - val_loss: 0.6934
Epo

In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Data2Vec_Large/train_24.npz')
test_data = np.load('./Data2Vec_Large/test_24.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 488ms/step - accuracy: 0.4816 - loss: 2.4451 - val_accuracy: 0.4186 - val_loss: 0.7133
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 479ms/step - accuracy: 0.5789 - loss: 0.6773 - val_accuracy: 0.4709 - val_loss: 0.7002
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 483ms/step - accuracy: 0.6531 - loss: 0.6505 - val_accuracy: 0.6047 - val_loss: 0.6812
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 484ms/step - accuracy: 0.6698 - loss: 0.6601 - val_accuracy: 0.5581 - val_loss: 0.6929
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 486ms/step - accuracy: 0.7043 - loss: 0.5681 - val_accuracy: 0.5930 - val_loss: 0.6644
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 481ms/step - accuracy: 0.7093 - loss: 0.5590 - val_accuracy: 0.6337 - val_loss: 0.6621
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 482ms/step - accuracy: 0.7722 - loss: 0.4740 - val_accuracy: 0.6337 - val_loss: 0.6464
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 483ms/step - accuracy: 0.8172 - loss: 0.4350 - val_accuracy: 0.639

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4947 - loss: 1.4807 - val_accuracy: 0.5407 - val_loss: 0.7254
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5145 - loss: 1.2771 - val_accuracy: 0.5814 - val_loss: 0.7070
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5457 - loss: 1.0310 - val_accuracy: 0.5233 - val_loss: 0.7050
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5385 - loss: 0.9062 - val_accuracy: 0.4593 - val_loss: 0.7238
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5560 - loss: 0.9194 - val_accuracy: 0.4302 - val_loss: 0.7095
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5462 - loss: 0.8580 - val_accuracy: 0.5349 - val_loss: 0.6955
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5421 - loss: 0.8223 - val_accuracy: 0.5640 - val_loss: 0.6792
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5803 - loss: 0.7844 - val_accuracy: 0.5349 - val_loss: 0.7015
Epo